In [45]:
import pandas as pd

In [46]:
df_accounts = pd.read_csv('data/accounts.csv')
df_alliance_ff = pd.read_csv('data/alliance_ff.csv')
df_alliance_membership = pd.read_csv('data/alliance_membership.csv')
df_chat_messages_1 = pd.read_csv('data/chat_messages_1.csv', dtype='unicode')
df_chat_messages_2 = pd.read_csv('data/chat_messages_2.csv', dtype='unicode')

Clean dataset chat_messages_1   

Drop unnecessary columns

In [47]:
columns_to_drop = ['risk', 'filter_detected_language', 'is_family_friendly', 'BULLYING', 'VIOLENCE', 'RELATIONSHIP_SEXUAL_CONTENT', 'VULGARITY', 'DRUGS_ALCOHOL', 'IN_APP', 'ALARM', 'FRAUD', 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS']
df_chat_messages_1.drop(columns_to_drop, inplace=True, axis=1)
df_chat_messages_1.to_csv("modified_chat_messages_1.csv", index=False)

Clean dataset chat_messages_2

Drop unnecessary columns

In [48]:
columns_to_drop2 = ['risk', 'filter_detected_language', 'is_family_friendly', 'BULLYING', 'VIOLENCE', 'RELATIONSHIP_SEXUAL_CONTENT', 'VULGARITY', 'DRUGS_ALCOHOL', 'IN_APP', 'ALARM', 'FRAUD', 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS']
df_chat_messages_2.drop(columns_to_drop2, inplace=True, axis=1)
df_chat_messages_2.to_csv("modified_chat_messages_2.csv", index=False)

Concat two csv

In [49]:
df_modified1 = pd.read_csv("modified_data/modified_chat_messages_1.csv", dtype='unicode')
df_modified2 = pd.read_csv("modified_data/modified_chat_messages_2.csv", dtype='unicode')

combined_df = pd.concat([df_modified1, df_modified2], ignore_index=True)
combined_df.to_csv("merged_message_data.csv", index=False)

Concat two csv

In [50]:
df_modified1 = pd.read_csv("modified_data/modified_chat_messages_1.csv", dtype='unicode')
df_modified2 = pd.read_csv("modified_data/modified_chat_messages_2.csv", dtype='unicode')

combined_df = pd.concat([df_modified1, df_modified2], ignore_index=True)
combined_df.to_csv("merged_message_data.csv", index=False)

Add offensive column

In [51]:
merged_df = pd.read_csv("merged_message_data.csv", dtype='unicode')
merged_df['GENERAL_RISK'] = pd.to_numeric(merged_df['GENERAL_RISK'], errors='coerce')
merged_df['offensive'] = (merged_df['GENERAL_RISK'] >= 5).astype(int)
merged_df.to_csv("offensive_added.csv", index=False)

HuggingFace Login

In [52]:
from huggingface_hub import notebook_login
notebook_login()

Train Model

In [14]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report

model_name = "cardiffnlp/twitter-roberta-base-offensive"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

data = pd.read_csv('offensive_added.csv',dtype='unicode', low_memory=False)
texts_column = data['raw_message'].head(100)
offensive_column = data['offensive'].head(100)

filtered_df = texts_column.dropna()
cleaned_df = filtered_df.str.contains(r"[^\w]", regex=True)

texts_column_list = cleaned_df.apply(lambda x: str(x)).tolist()
offensive_column_list = offensive_column.tolist()

X_train, X_test, y_train, y_test = train_test_split(texts_column_list, offensive_column_list, test_size = 0.20, random_state = 0)

train_encodings = tokenizer(X_train, truncation=False, padding=True)
val_encodings = tokenizer(X_test, truncation=False, padding=True)

class OffensiveDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = OffensiveDataset(train_encodings, y_train)
test_dataset = OffensiveDataset(val_encodings, y_test)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated  Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

predictions = trainer.predict(test_dataset)

from sklearn.metrics import accuracy_score

test_labels = torch.tensor(y_test)  # Convert test labels to tensor
accuracy = accuracy_score(test_labels, predictions.predictions)
print(f"Test Accuracy: {accuracy:.4f}")


TypeError: new(): invalid data type 'str'